<a href="https://colab.research.google.com/github/fastai-energetic-engineering/ashrae/blob/master/_notebooks/2021-06-27-Getting-ASHRAE-Energy-Prediction-Data-from-Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Kaggle Data for ASHRAE Energy Prediction
> "How to download Kaggle data from Colab."

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [kaggle, preprocessing]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: false


In [4]:
#collapse
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [69]:
#collapse
from fastbook import *
import os
from google.colab import files
import pandas as pd

This notebook demonstrates how I downloaded the [ASHRAE Energy Prediction Data](https://www.kaggle.com/c/ashrae-energy-prediction/overview) from Kaggle.

First, we need to install the [Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials).

In [3]:
!pip install kaggle --upgrade -q

I will download the data into a folder in my google drive. First, I will set my home directory.

In [16]:
p = Path('gdrive/MyDrive/Colab Notebooks/ashrae/')
os.chdir(p) # change directory

We need to download Kaggle API token and then put the `.json` file in `.kaggle` folder. We can upload the key directly from colab.

In [ ]:
files.upload() # use this to upload your API json key
!mkdir ~/.kaggle # create folder
!cp kaggle.json ~/.kaggle/ # move the key into the folder
!chmod 600 ~/.kaggle/kaggle.json # change permissions of the file

We can finally download the file!

In [61]:
os.chdir('data') # move to data folder
!kaggle competitions download -c ashrae-energy-prediction

In [66]:
# extract zip files then remove the .zip
for item in os.listdir(): # for every item in the folder
    if item.endswith('.zip'): # check if it is a .zip file
        file_extract(item) # if it is, then extract file
        os.remove(item) # and then remove the .zip

In [67]:
os.chdir("..") # return to initial folder

## Joining Tables

Our training data comprised of three tables:
- `building_metadata.csv`
- `weather_train.csv`
- `train.csv`

We need to join the tables. First, let's see what's in the tables.

In [70]:
building = pd.read_csv('data/building_metadata.csv')
weather = pd.read_csv('data/weather_train.csv')
train = pd.read_csv('data/train.csv')

`building` contains the buildings' metadata.

In [71]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


- `site_id` - Foreign key for the weather files.
- `building_id` - Foreign key for training.csv
- `primary_use` - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
- `square_feet` - Gross floor area of the building
- `year_built` - Year building was opened
- `floor_count` - Number of floors of the building

`weather` contains weather data from the closest meteorological station.

In [72]:
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


- `site_id`
- `air_temperature` - Degrees Celsius
- `cloud_coverage` - Portion of the sky covered in clouds, in oktas
- `dew_temperature` - Degrees Celsius
- `precip_depth_1_hr` - Millimeters
- `sea_level_pressure` - Millibar/hectopascals
- `wind_direction` - Compass direction (0-360)
- `wind_speed` - Meters per second

Finally, `train` contains the target variable, `meter reading`, which represents energy consumption in kWh.

In [73]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


- `building_id` - Foreign key for the building metadata.
- `meter` - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}. Not every building has all meter types.
- `timestamp` - When the measurement was taken
- `meter_reading` - The target variable. Energy consumption in kWh (or equivalent).